In [7]:
# Initialize
import simpy
import random

SIM_TIME = 50
PASSENGER_INTERVAL = 10
NUM_FLOORS = 3
MAX_CAPACITY = 4

In [ ]:
def get_arrival_rate(current_time_minutes):
    """
    Returns the arrival rate (passengers per minute) based on time of day.
    Peak times:
    - 7:30-8:30 AM (450-510 minutes)
    - 12:00-1:00 PM (720-780 minutes) 
    - 5:00-10:00 PM (1020-1320 minutes)
    """
    
    # Convert time to minutes from midnight
    time_of_day = current_time_minutes % 1440
    
    # Morning rush: 7:30-8:30 AM
    if 450 <= time_of_day <= 510:
        return 0.5  # High arrival rate (1 passenger every 2 minutes on average)
    
    # Lunch rush: 12:00-1:00 PM
    elif 720 <= time_of_day <= 780:
        return 0.4  # High arrival rate (1 passenger every 2.5 minutes on average)
    
    # Evening rush: 5:00-10:00 PM
    elif 1020 <= time_of_day <= 1320:
        return 0.3  # Medium-high arrival rate (1 passenger every 3.3 minutes on average)
    
    # Off-peak hours
    else:
        return 0.05  # Low arrival rate (1 passenger every 20 minutes on average)

def time_to_string(minutes):
    """Convert minutes from midnight to HH:MM format"""
    hours = int(minutes // 60) % 24
    mins = int(minutes % 60)
    return f"{hours:02d}:{mins:02d}"


In [8]:
class Passenger:
    def __init__(self, pid, pickup, destination):
        self.id = pid
        self.pickup = pickup
        self.destination = destination
        # should add start time in system so we can track how long they are in the system
    
    def __repr__(self, color = '\033[97m'):         # changes representation of passenger to be easier to read
        return f"{color}P{self.id}({self.pickup} --> {self.destination})"

I want to change passenger_generator a little bit. 
This now has all passengers arriving at a rate of 1 / lambda. 
    The issue is floor 1 will have many more people arriving than floor 2, 3, ....
    I think it would be reasonable to have X people arriving at floor 1 and then
        floor 2 and 3 would have X/2 or if we have 5 floors each would have X / 4. 
            - Cree

In [ ]:
#Passenger Generator
def passenger_generator(env, elevator):
    pid = 1         #start with pid 1
    while True:
        # Get current arrival rate based on time of day
        current_rate = get_arrival_rate(env.now)
        
        # Calculate inter-arrival time using exponential distribution
        if current_rate > 0:
            inter_arrival_time = random.expovariate(current_rate)
        else:
            inter_arrival_time = 60  # Default to 1 hour if rate is 0
        
        yield env.timeout(inter_arrival_time)
        
        pickup = random.randint(0, 2)           # random floor
        destination = random.randint(0,2)       # random destination
        while pickup == destination:
            destination = random.randint(0,2)
        
        passenger = Passenger(pid, pickup, destination) # create the passenger
        time_str = time_to_string(env.now)
        print(f"[{time_str} ({env.now:.1f})] Passenger {pid} arrived, travel plan:{pickup}-->{destination}")
        elevator.request_pickup(passenger)              # request pickup from elevator
        pid +=1


Elevator currently has a log, pickup_requests, and a run function that onboards and offboards people. It, however, doesn't move up or down floors ¯\_(ツ)_/¯

To start, I think we should go with a super simple:
    if self.floor < num_floors:
        floor += 1
    else: floor -= 1
    that way it goes all the way up and then goes all the way down. picking people up and dropping them off as it goes. 

In [ ]:
class Elevator:
    def __init__(self, env, num_floors):
        self.env = env
        self.floor = 1          # current floor 
        self.direction = 'up'  # up or down
        self.num_floors = num_floors # total
        self.pickup_requests = {floor: [] for floor in range(num_floors)}
        self.onboard = []

    def log(self, message):
        print(f"    [{self.env.now:.1f}] Elevator floor: {self.floor} | {message}")

    def request_pickup(self, passenger):
        self.pickup_requests[passenger.pickup].append(passenger)
        self.log(f"Passenger {passenger.id} requests a pickup at floor {passenger.pickup}")

    def run_elevator(self):         # elevator logic 
        while True:
            offboarding = [p for p in self.onboard if p.destination == self.floor]
            for p in offboarding:
                self.onboard.remove(p)
                print(f"Passengers left: {self.onboard}")
            boarding = [p for p in self.pickup_requests[self.floor]]
            for p in boarding:
                self.onboard.add(p)

In [ ]:
    def run_elevator(self):
        while True:
            # 1) Off-load and on-board as before…
            offboarding = [p for p in self.onboard if p.destination == self.floor]
            for p in offboarding:
                self.onboard.remove(p)
                self.log(f"Dropped off {p}")

            boarding = self.pickup_requests[self.floor]
            for p in boarding:
                self.onboard.append(p)
                self.log(f"Picked up  {p}")
            self.pickup_requests[self.floor].clear()

            # 2) Move one floor in the current direction
            if self.direction == 'up':
                if self.floor < self.num_floors - 1:
                    self.floor += 1
                else:
                    # hit top → reverse
                    self.direction = 'down'
                    self.floor -= 1
            else:  # direction == 'down'
                if self.floor > 0:
                    self.floor -= 1
                else:
                    # hit bottom → reverse
                    self.direction = 'up'
                    self.floor += 1

            self.log(f"Moving {self.direction} to floor {self.floor}")

            # 3) Wait for travel time (e.g. 1 time-unit per floor)
            yield self.env.timeout(1)

In [11]:
random.seed(23)
env = simpy.Environment()
elevator = Elevator(env, num_floors = NUM_FLOORS)
env.process(passenger_generator(env, elevator))
env.run(until = SIM_TIME)


[25.9] Passenger 1 arrived, travel plan:1-->0
    [25.9] Elevator floor: 1 | Passenger 1 requests a pickup at floor 1
[26.1] Passenger 2 arrived, travel plan:1-->2
    [26.1] Elevator floor: 1 | Passenger 2 requests a pickup at floor 1
[30.5] Passenger 3 arrived, travel plan:2-->0
    [30.5] Elevator floor: 1 | Passenger 3 requests a pickup at floor 2
[33.6] Passenger 4 arrived, travel plan:0-->2
    [33.6] Elevator floor: 1 | Passenger 4 requests a pickup at floor 0
[39.7] Passenger 5 arrived, travel plan:0-->2
    [39.7] Elevator floor: 1 | Passenger 5 requests a pickup at floor 0
[46.6] Passenger 6 arrived, travel plan:0-->2
    [46.6] Elevator floor: 1 | Passenger 6 requests a pickup at floor 0


In [ ]:
##  Test Cell   ##
##              ##
##################


p = Passenger(1, 0, 2)          # test passenger
elevator.request_pickup(p)      # request pickup for test
print(elevator.pickup_requests) # print all pickup_requests. 
elevator.onboard.append(p)      # add them 
print(elevator.onboard)         # test if they're on
elevator.onboard.remove(p)      #remove them
print("passengers left: ", elevator.onboard)

elevator.direction

    [50.0] Elevator floor: 1 | Passenger 1 requests a pickup at floor 0
{0: [P4(0 --> 2), P5(0 --> 2), P6(0 --> 2), P1(0 --> 2), P1(0 --> 2), P1(0 --> 2)], 1: [P1(1 --> 0), P2(1 --> 2)], 2: [P3(2 --> 0)]}
[P1(0 --> 2)]
passengers left:  []
up


'up'